In [22]:
import urllib2
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import os
import sys

In [23]:
#done:'51004','51002','51003','51101','51001','51000','51208'
#done:'51201','51204','51210','51202','51203','51205','51407','51206'
#done:'51wh0','nwwh1','oouh1','mokh1','klih1','kwhh1','iloh1'
#51407 and 51wh0 don't have standard meterological data

#if the buoy's name has letter, it has to be lower letter.
readin = []

In [24]:
hist_data_url = "http://www.ndbc.noaa.gov/historical_data.shtml"
main_url = "http://www.ndbc.noaa.gov"
data_folder = "python_test/metocean_data"

In [25]:
def readin_ndbc_stdmet(name_buoy):
    
    #save all years' links of the buoy in to 'a_tags'
    req_1 = urllib2.urlopen(hist_data_url)
    soup_1 = BeautifulSoup(req_1)
    a_tags = soup_1.find_all("a",attrs = {'href':re.compile(".*"+name_buoy+
                                                          ".*stdmet.*")})    
    if len(a_tags) == 0:
        print "there is no historical standard meterological data for %s buoy"\
               %(name_buoy)
        return
    print "number of years of the data for %s buoy is %i" \
           %(name_buoy, len(a_tags))
        
    for i in range(len(a_tags)):
        
        #for each link, open the 2nd page which contains the download link 
        link = a_tags[i].get('href')
        req_2 = urllib2.urlopen(main_url+link)
        soup_2 = BeautifulSoup(req_2)
        
        #locate the download link and open the .txt file into 'data_str'
        data_tag = soup_2.find_all("a",attrs = {'href':re.compile(".*"+name_buoy\
                                                +".*txt.*stdmet.*")})
        data_link = data_tag[0].get('href')
        req_3 = urllib2.urlopen(main_url+data_link)
        data_str = req_3.read()
        
        #write data into .csv file and save to the existing folder "data_folder"
        if sys.version_info[0]<3:
            from StringIO import StringIO
        else:
            from io import StringIO
        data_csv = StringIO(data_str)
        data_df = pd.read_csv(data_csv,delim_whitespace = True)
        year = data_df.iat[2,0]
        file_path = '/'+data_folder+'/'+name_buoy+'/'+str(year)+'.csv'
        directory = os.path.dirname(file_path)       
        try:
            os.stat(directory)
        except:
            os.mkdir(directory)            
        data_df.to_csv(file_path)
        time.sleep(1)
    print "%s buoy is done" %name_buoy
    return    

In [26]:
for i in range(len(readin)):
    readin_ndbc_stdmet(readin[i])
    if i in (3,6,9,12,15,18,21):
        time.sleep(10)
    time.sleep(3)

number of years of the data for oouh1 buoy is 12
oouh1 buoy is done
